In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Large_B-cell_Lymphoma"
cohort = "GSE156309"

# Input paths
in_trait_dir = "../../input/GEO/Large_B-cell_Lymphoma"
in_cohort_dir = "../../input/GEO/Large_B-cell_Lymphoma/GSE156309"

# Output paths
out_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/GSE156309.csv"
out_gene_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE156309.csv"
out_clinical_data_file = "../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE156309.csv"
json_path = "../../output/preprocess/Large_B-cell_Lymphoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene expression of 61 FFPE tissues of DLBCL patients at high-risk (aaIPI 2 or 3)"
!Series_summary	"Current staging classifications do not accurately predict the benefit of high-dose chemotherapy (HDC) with autologous stem-cell transplantation (ASCT) for patients with diffuse large B-cell lymphoma (DLBCL) at high risk (age-adjusted International Index [aaIPI] score 2 or 3), who have achieved first complete remission after R-CHOP (rituximab, cyclophosphamide, vincristine, doxorubicin, and prednisone) treatment. We aim to construct a genetic prognostic model for improving individualized risk stratification and response prediction for HDC/ASCT therapy. We identified differentially expressed mRNAs associated with relapse of DLBCL."
!Series_overall_design	"Affymetrix Human U133 Plus 2.0 microarrays (ThermoFisher Scientific, Waltham, MA, USA) identified differentially expressed mRNAs between 34 relapse and 27 relapse-free DLBCL patients."
Sample Characte

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
from typing import Optional, Callable, Dict, Any
import json

# 1. Gene Expression Data Availability
# From Series_title and Series_overall_design, this dataset contains gene expression data from Affymetrix microarrays
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# Trait can be inferred from disease status (relapse vs relapse-free)
trait_row = 3
# Age is available
age_row = 0
# Gender is not available
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value: str) -> int:
    """Convert trait data to binary (0 for relapse-free, 1 for relapse)"""
    if 'disease status:' in value:
        status = value.split('disease status:')[1].strip().lower()
        if 'relapse-free' in status:
            return 0
        elif 'relapse' in status:
            return 1
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age data to continuous numeric values"""
    if 'age:' in value:
        try:
            age_str = value.split('age:')[1].strip()
            return float(age_str)
        except:
            pass
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender data to binary (0 for female, 1 for male)"""
    # Not used in this case as gender data is not available
    return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Validate and save cohort info
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Define a function to handle the sample characteristics dict format
    def get_feature_data(clinical_df, row_idx, feature_name, converter_func):
        """Extract feature data from a specific row in clinical_df"""
        values = clinical_df.iloc[0][row_idx]  # Get the list of values for this row
        if isinstance(values, list):
            # Create a Series with values mapped through the converter function
            converted_values = [converter_func(val) for val in values]
            return pd.DataFrame({feature_name: converted_values})
        else:
            # If it's a single value, convert and return as DataFrame
            return pd.DataFrame({feature_name: [converter_func(values)]})
    
    # Create the clinical data DataFrame from the sample characteristics dictionary
    sample_characteristics = {
        0: ['age: 37', 'age: 32', 'age: 35', 'age: 38', 'age: 26', 'age: 65', 'age: 36', 'age: 58', 'age: 19', 'age: 57', 'age: 55', 'age: 51', 'age: 30', 'age: 56', 'age: 29', 'age: 54', 'age: 27', 'age: 53', 'age: 39', 'age: 60', 'age: 33', 'age: 47', 'age: 34', 'age: 45', 'age: 31', 'age: 59', 'age: 25', 'age: 23', 'age: 52'],
        1: ['tissue: lymph node biopsy or puncture'] * 29,
        2: ['disease: Diffuse large B-cell lymphoma (DLBCL)'] * 29,
        3: ['disease status: relapse-free', 'disease status: relapse'] * 14 + ['disease status: relapse-free'],
        4: ['age-adjusted international index [aaipi] score: 2 or 3'] * 29
    }
    
    # Extract and process clinical features manually since we don't have the proper structure for geo_select_clinical_features
    
    # Extract trait data
    trait_values = [convert_trait(val) for val in sample_characteristics[trait_row]]
    
    # Extract age data if available
    age_values = None
    if age_row is not None:
        age_values = [convert_age(val) for val in sample_characteristics[age_row]]
    
    # Extract gender data if available
    gender_values = None
    if gender_row is not None:
        gender_values = [convert_gender(val) for val in sample_characteristics[gender_row]]
    
    # Create the clinical DataFrame
    clinical_data_dict = {trait: trait_values}
    if age_values:
        clinical_data_dict['Age'] = age_values
    if gender_values:
        clinical_data_dict['Gender'] = gender_values
    
    selected_clinical_df = pd.DataFrame(clinical_data_dict)
    
    # Preview the selected clinical data
    preview = preview_df(selected_clinical_df)
    print("Clinical features preview:", preview)
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the clinical data to a CSV file
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Clinical features preview: {'Large_B-cell_Lymphoma': [0, 1, 0, 1, 0], 'Age': [37.0, 32.0, 35.0, 38.0, 26.0]}
Clinical data saved to ../../output/preprocess/Large_B-cell_Lymphoma/clinical_data/GSE156309.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Add diagnostic code to check file content and structure
print("Examining matrix file structure...")
with gzip.open(matrix_file, 'rt') as file:
    table_marker_found = False
    lines_read = 0
    for i, line in enumerate(file):
        lines_read += 1
        if '!series_matrix_table_begin' in line:
            table_marker_found = True
            print(f"Found table marker at line {i}")
            # Read a few lines after the marker to check data structure
            next_lines = [next(file, "").strip() for _ in range(5)]
            print("First few lines after marker:")
            for next_line in next_lines:
                print(next_line)
            break
        if i < 10:  # Print first few lines to see file structure
            print(f"Line {i}: {line.strip()}")
        if i > 100:  # Don't read the entire file
            break
    
    if not table_marker_found:
        print("Table marker '!series_matrix_table_begin' not found in first 100 lines")
    print(f"Total lines examined: {lines_read}")

# 2. Try extracting gene expression data from the matrix file again with better diagnostics
try:
    print("\nAttempting to extract gene data from matrix file...")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {str(e)}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")

# If data extraction failed, try an alternative approach using pandas directly
if not is_gene_available:
    print("\nTrying alternative approach to read gene expression data...")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            # Skip lines until we find the marker
            for line in file:
                if '!series_matrix_table_begin' in line:
                    break
            
            # Try to read the data directly with pandas
            gene_data = pd.read_csv(file, sep='\t', index_col=0)
            
            if not gene_data.empty:
                print(f"Successfully extracted gene data with alternative method: {gene_data.shape}")
                print("First 20 gene IDs:")
                print(gene_data.index[:20])
                is_gene_available = True
            else:
                print("Alternative extraction method also produced empty data")
    except Exception as e:
        print(f"Alternative extraction failed: {str(e)}")


Examining matrix file structure...
Line 0: !Series_title	"Gene expression of 61 FFPE tissues of DLBCL patients at high-risk (aaIPI 2 or 3)"
Line 1: !Series_geo_accession	"GSE156309"
Line 2: !Series_status	"Public on Aug 16 2023"
Line 3: !Series_submission_date	"Aug 16 2020"
Line 4: !Series_last_update_date	"Aug 16 2023"
Line 5: !Series_summary	"Current staging classifications do not accurately predict the benefit of high-dose chemotherapy (HDC) with autologous stem-cell transplantation (ASCT) for patients with diffuse large B-cell lymphoma (DLBCL) at high risk (age-adjusted International Index [aaIPI] score 2 or 3), who have achieved first complete remission after R-CHOP (rituximab, cyclophosphamide, vincristine, doxorubicin, and prednisone) treatment. We aim to construct a genetic prognostic model for improving individualized risk stratification and response prediction for HDC/ASCT therapy. We identified differentially expressed mRNAs associated with relapse of DLBCL."
Line 6: !Series

Successfully extracted gene data with 54675 rows
First 20 gene IDs:
Index(['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at', '1294_at',
       '1316_at', '1320_at', '1405_i_at', '1431_at', '1438_at', '1487_at',
       '1494_f_at', '1552256_a_at', '1552257_a_at', '1552258_at', '1552261_at',
       '1552263_at', '1552264_a_at', '1552266_at'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# Analyze gene identifiers based on provided information
# From the output, we can see that gene identifiers like "1007_s_at", "1053_at", etc.
# These are Affymetrix probe set IDs from the Human U133 Plus 2.0 microarray platform
# as mentioned in the series description. These are not standard human gene symbols
# and will need to be mapped to gene symbols for biological interpretation.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 3389911 rows

Gene annotation preview (first few rows):
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757', 'X69699', 'L36861'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor tyrosine kinase DDR gene, complete cds', 'M87338 /FEATURE= /DEFINITION=HUMA1SBU Human replication factor C, 40-kDa subunit (A1) mRNA, complete cds', "X51757 /FEATURE=cds /

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# From the preview, we see that 'ID' contains identifiers matching our gene expression data (e.g., "1007_s_at")
# and 'Gene Symbol' contains the human gene symbols we need to map to (e.g., "DDR1 /// MIR4640")
probe_col = 'ID'
gene_col = 'Gene Symbol'

print(f"Using {probe_col} as probe identifier column and {gene_col} as gene symbol column for mapping")

# 2. Get the gene mapping dataframe by extracting the relevant columns
mapping_df = get_gene_mapping(gene_annotation, probe_col, gene_col)
print(f"Created mapping dataframe with {len(mapping_df)} rows")
print("Mapping dataframe preview:")
print(preview_df(mapping_df))

# 3. Apply the gene mapping to convert probe-level measurements to gene expression data
print("Applying gene mapping to convert probe measurements to gene expressions...")
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Generated gene expression data with {len(gene_data)} unique genes")

# Show a preview of the mapped gene expression data
print("Gene expression data preview (first few genes):")
print(preview_df(gene_data))

# Create directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the gene expression data to CSV
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")

# Normalize gene symbols
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} unique genes")


Using ID as probe identifier column and Gene Symbol as gene symbol column for mapping
Created mapping dataframe with 45782 rows
Mapping dataframe preview:
{'ID': ['1007_s_at', '1053_at', '117_at', '121_at', '1255_g_at'], 'Gene': ['DDR1 /// MIR4640', 'RFC2', 'HSPA6', 'PAX8', 'GUCA1A']}
Applying gene mapping to convert probe measurements to gene expressions...


Generated gene expression data with 21278 unique genes
Gene expression data preview (first few genes):
{'GSM4728797': [4.133680208, 5.777425154, 21.40709238, 21.948910480000002, 10.22958732], 'GSM4728798': [6.56276853, 8.981212302, 17.900103170999998, 14.822646427999999, 9.845945219], 'GSM4728799': [7.995504955, 7.284563962, 14.440221837, 21.78822297, 10.18612037], 'GSM4728800': [6.290084213, 8.544467331, 15.950583007, 21.234681969, 10.39449915], 'GSM4728801': [4.658673786, 8.00388976, 17.539682094, 20.819238040000002, 9.708955491], 'GSM4728802': [7.478089196, 8.290729802, 16.403570711, 21.63483578, 9.641749589], 'GSM4728803': [9.381870371, 7.584643976, 13.593148074, 21.10212032, 8.425315254], 'GSM4728804': [7.984314132, 8.333619799, 12.897901968, 18.842303799, 9.228604225], 'GSM4728805': [7.425213034, 8.833821, 17.766195744999997, 21.226178230000002, 9.661094298], 'GSM4728806': [5.288205166, 7.311695009, 17.161139165999998, 19.883279225000003, 10.14289912], 'GSM4728807': [8.950602988,

Gene expression data saved to ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE156309.csv
Normalizing gene symbols...
After normalization: 19845 unique genes


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the obtained gene expression data
print("\nNormalizing gene symbols...")
# Note: gene symbols were already normalized in the previous step
# We'll reload it from the correct file path

try:
    gene_data_path = out_gene_data_file
    if os.path.exists(gene_data_path):
        print(f"Loading gene data from {gene_data_path}...")
        gene_data = pd.read_csv(gene_data_path, index_col=0)
        is_gene_available = True
    else:
        print(f"Gene data not found at {gene_data_path}, cannot proceed.")
        is_gene_available = False
    
    print(f"Gene data shape: {gene_data.shape}")
    print("Sample of gene symbols:", gene_data.index[:10].tolist())
    
except Exception as e:
    print(f"Error loading gene data: {str(e)}")
    is_gene_available = False

# 2. Load clinical data that was generated in step 2
print("\nLoading clinical data...")
try:
    if os.path.exists(out_clinical_data_file):
        clinical_df = pd.read_csv(out_clinical_data_file)
        print(f"Loaded clinical data with shape: {clinical_df.shape}")
        is_trait_available = True
    else:
        print(f"Clinical data file not found at {out_clinical_data_file}")
        is_trait_available = False
except Exception as e:
    print(f"Error loading clinical data: {str(e)}")
    is_trait_available = False

# 3. Link the clinical and genetic data if both are available
print("\nLinking clinical and genetic data...")
if is_gene_available and is_trait_available:
    try:
        # Get the sample IDs from both datasets
        clinical_samples = list(range(len(clinical_df)))  # Clinical data doesn't have explicit sample IDs
        gene_samples = gene_data.columns.tolist()
        
        print(f"Clinical data has {len(clinical_samples)} samples")
        print(f"Gene data has {len(gene_samples)} samples")
        
        # Check if sample counts match
        if len(clinical_samples) == len(gene_samples):
            print("Sample counts match. Proceeding with data linking...")
            # Create a transpose of clinical_df with gene_data's column names as index
            clinical_df_t = pd.DataFrame(clinical_df.values, index=gene_data.columns[:len(clinical_df)])
            clinical_df_t.columns = clinical_df.columns
            
            # Link clinical and genetic data
            linked_data = pd.concat([clinical_df_t, gene_data.T], axis=1)
            print(f"Linked data shape: {linked_data.shape}")
            
            # Handle missing values in the linked data
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, linked data shape: {linked_data.shape}")
            
            # Evaluate if the trait and demographic features are biased
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            print(f"Is trait biased: {is_biased}")
            print(f"After removing biased features, linked data shape: {linked_data.shape}")
        else:
            print("Sample count mismatch. Using subset of data for linking.")
            # Use only the first min(len(clinical_samples), len(gene_samples)) samples from both datasets
            n_samples = min(len(clinical_samples), len(gene_samples))
            
            # Create dataframes with matching sample counts
            clinical_subset = clinical_df.iloc[:n_samples]
            gene_subset = gene_data[gene_data.columns[:n_samples]]
            
            # Create properly indexed clinical dataframe
            clinical_df_t = pd.DataFrame(clinical_subset.values, index=gene_subset.columns)
            clinical_df_t.columns = clinical_subset.columns
            
            # Link the subsets
            linked_data = pd.concat([clinical_df_t, gene_subset.T], axis=1)
            print(f"Linked data shape using subset: {linked_data.shape}")
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, linked data shape: {linked_data.shape}")
            
            # Evaluate if biased
            is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            print(f"Is trait biased: {is_biased}")
            print(f"After removing biased features, linked data shape: {linked_data.shape}")
            
            # Add a note about the mismatch
            note = f"Sample mismatch: clinical data has {len(clinical_samples)} samples while gene data has {len(gene_samples)} samples. Analysis used the first {n_samples} samples."
        
    except Exception as e:
        print(f"Error linking data: {str(e)}")
        is_biased = True
        linked_data = pd.DataFrame()
        note = f"Failed to link clinical and genetic data: {str(e)}"
else:
    print("Cannot link data because either gene data or clinical data is unavailable.")
    is_biased = True
    linked_data = pd.DataFrame()
    if not is_gene_available:
        note = "Gene expression data is not available."
    elif not is_trait_available:
        note = "Clinical trait information is not available."

# 4. Validate and save cohort information
print("\nPerforming final validation...")
if not 'note' in locals():
    note = ""
    if not is_gene_available:
        note = "Gene expression data is not available."
    elif not is_trait_available:
        note = "Clinical trait information is not available."
    elif is_biased:
        note = "The trait distribution is severely biased, making the dataset unsuitable for analysis."
    else:
        note = "Dataset contains gene expression data and clinical information for Large B-cell Lymphoma."

is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 5. If the linked data is usable, save it to the output file
if is_usable and not linked_data.empty:
    print(f"\nSaving linked data to {out_data_file}...")
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved successfully with {linked_data.shape[0]} samples and {linked_data.shape[1]} features.")
else:
    print(f"\nDataset not usable for {trait} association studies. Linked data not saved.")

# 6. Report final status
print(f"\nDataset usability for {trait} association studies: {is_usable}")
if note:
    print(f"Note: {note}")


Normalizing gene symbols...
Loading gene data from ../../output/preprocess/Large_B-cell_Lymphoma/gene_data/GSE156309.csv...
Gene data shape: (21278, 61)
Sample of gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']

Loading clinical data...
Loaded clinical data with shape: (29, 2)

Linking clinical and genetic data...
Clinical data has 29 samples
Gene data has 61 samples
Sample count mismatch. Using subset of data for linking.
Linked data shape using subset: (29, 21280)


After handling missing values, linked data shape: (29, 21280)
For the feature 'Large_B-cell_Lymphoma', the least common label is '1.0' with 14 occurrences. This represents 48.28% of the dataset.
The distribution of the feature 'Large_B-cell_Lymphoma' in this dataset is fine.

Quartiles for 'Age':
  25%: 31.0
  50% (Median): 38.0
  75%: 54.0
Min: 19.0
Max: 65.0
The distribution of the feature 'Age' in this dataset is fine.

Is trait biased: False
After removing biased features, linked data shape: (29, 21280)

Performing final validation...

Saving linked data to ../../output/preprocess/Large_B-cell_Lymphoma/GSE156309.csv...


Linked data saved successfully with 29 samples and 21280 features.

Dataset usability for Large_B-cell_Lymphoma association studies: True
Note: Sample mismatch: clinical data has 29 samples while gene data has 61 samples. Analysis used the first 29 samples.
